In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
import random
import dataclasses
import jax
import optax

import jax.numpy as jnp
import numpy as np

from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from penzai.data_effects.side_output import SideOutputValue
from micrlhf.utils.activation_manipulation import add_vector
from micrlhf.utils.load_sae import get_sae
from functools import partial

In [3]:
filename = "models/phi-3-16.gguf"
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(filename, device_map="auto")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from task_vector_utils import load_tasks, ICLDataset, ICLSequence
tasks = load_tasks()

Cloning into 'itv'...
fatal: unable to access 'https://github.com/roeehendel/icl_task_vectors data/itv/': URL using bad/illegal format or missing URL


In [6]:
from micrlhf.llama import LlamaBlock
from micrlhf.sampling import sample, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)

In [7]:
def tokenized_to_inputs(input_ids, attention_mask):
    token_array = jnp.asarray(input_ids)
    token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")

    mask_array = jnp.asarray(attention_mask, dtype=jnp.bool)
    mask_array = jax.device_put(mask_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    mask_array = pz.nx.wrap(mask_array, "batch", "seq").untag("batch").tag("batch")

    inputs = llama.inputs.from_basic_segments(token_array)
    return inputs

In [8]:
prompt = "<user>Follow the pattern\n{}"

In [9]:

target_layer = 17

In [10]:
task_names = [
    "en_es"
]

task = "en_es"

n_seeds = 10

# n_few_shots, batch_size, max_seq_len = 64, 64, 512
n_few_shots, batch_size, max_seq_len = 32, 64, 256

In [11]:
def get_logprob_diff(logits: jnp.ndarray, target_tokens, print_results=False):
    logprobs = jax.nn.log_softmax(logits, axis=-1)
    answer_logprobs = logprobs[:, -1]

    target_logprobs = jnp.take_along_axis(answer_logprobs, target_tokens[:, None], axis=-1).squeeze()

    if print_results:
        print(
            tokenizer.decode(answer_logprobs.argmax(axis=-1))
        )

        print(
            tokenizer.decode(target_tokens)
        )

    return target_logprobs


In [12]:
from micrlhf.llama import LlamaBlock
from functools import partial

def make_get_resids(llama, layer_target):
    get_resids = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer_target
                                                                              ).apply(lambda x:
        pz.nn.Sequential([
            pz.de.TellIntermediate.from_config(tag=f"resid_pre"),
            x
        ])
    )
    get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
    return get_resids

In [13]:
import dataclasses
def get_loss(weights, dictionary, inputs, target_tokens, taker, initial_resids):
    weights = jax.nn.relu(weights)
    

    recon = jnp.einsum("fv,f->v", dictionary, weights)
    recon = recon.astype('bfloat16')

    modified = pz.nx.nmap(lambda a, b: a.at[-1].add(b))(
        initial_resids.untag("seq", "embedding"), pz.nx.wrap(recon, "embedding").untag("embedding")
        ).tag("seq", "embedding")

    inputs = dataclasses.replace(inputs, tokens=modified)

    logits = taker(inputs).unwrap("batch", "seq", "vocabulary")

    logprob_diff = get_logprob_diff(logits, target_tokens)
    loss = -logprob_diff.mean()

    return loss + 2e-2 * jnp.linalg.norm(weights, ord=1), ((weights != 0).sum(), loss)
    # return loss, ((weights != 0).sum(), loss)

In [14]:
def train_step(weights, opt_state, dictionary, inputs, target_tokens, taker, initial_resids, optimizer, lwg, pos_only=True):
    (loss, (l0, loss_)), grad = lwg(weights, dictionary, inputs, target_tokens, taker, initial_resids)

    updates, opt_state = optimizer.update(grad, opt_state, weights)
    weights = optax.apply_updates(weights, updates)
    # weights_abs = jnp.abs(weights)
    # weights = jnp.sign(weights) * jax.nn.relu(weights_abs - shrinkage)

    return loss, weights, opt_state, dict(l0=l0, loss=loss_)

In [15]:
jittify = lambda x: partial(jax.jit(lambda lr, *args, **kwargs: lr(*args, **kwargs)[1][0].value), x)

In [16]:
from task_vector_utils import ICLRunner, logprob_loss, make_act_adder, get_tv

class FeatureSearch:
    def __init__(self, task, pairs, target_layer, batch_size=32, n_shot=20, early_stopping_steps=50, max_seq_len=256, iterations=2000, seed=9, l1_coeff=1e-2, lr=3e-2, init_w=0.1, sae_v=6, n_first=2):
        self.task = task
        self.target_layer = target_layer
        self.sae = get_sae(target_layer, sae_v)
        self.seed = seed
        self.early_stopping_steps = early_stopping_steps
        self.iterations = iterations
        self.l1_coeff = l1_coeff
        self.lr = lr
        self.init_w = init_w
        self.n_shot = n_shot
        self.batch_size = batch_size
        self.n_first = n_first

        self.runner = ICLRunner(task, pairs, batch_size=batch_size, n_shot=n_shot, max_seq_len=max_seq_len, seed=seed)
        
        self.train_inputs = tokenized_to_inputs(
            **self.runner.get_tokens(self.runner.train_pairs, tokenizer)
        )

        self.eval_inputs = tokenized_to_inputs(
            **self.runner.get_tokens(self.runner.eval_pairs, tokenizer)
        )

        self.eval_tokens = self.runner.get_tokens(self.runner.eval_pairs, tokenizer)["input_ids"]

        self.initial_resids = self.get_initial_resids(self.eval_inputs)

        self.lwg = jax.value_and_grad(self.get_loss, has_aux=True)
        self.taker = self.make_taker()
    
    def get_initial_resids(self, inputs):
        get_resids_initial = make_get_resids(llama, self.target_layer)
        get_resids_initial = jittify(get_resids_initial)

        initial_resids = get_resids_initial(inputs)
        return initial_resids

    def make_taker(self):
        taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
            lambda i, x: x if i >= self.target_layer else pz.nn.Identity()
        ).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                        .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

        return taker
    
    def get_loss(self, weights):
        weights = jax.nn.relu(weights)

        recon = jnp.einsum("fv,f->v", self.sae["W_dec"], weights)
        recon = recon.astype('bfloat16')

        mask = self.eval_tokens == 1599
        positions = jnp.argwhere(mask)[:, -1]
        resids = self.initial_resids.unwrap("batch", "seq", "embedding")


        modified = jax.vmap(lambda a, b: a.at[b].add(recon))(
            resids, positions
        )
        modified = pz.nx.wrap(modified, "batch", "seq", "embedding")

        inputs = dataclasses.replace(self.eval_inputs, tokens=modified)
        logits = self.taker(inputs).unwrap("batch", "seq", "vocabulary")
        loss = logprob_loss(logits, self.eval_tokens, n_first=self.n_first)

        return loss + self.l1_coeff * jnp.linalg.norm(weights, ord=1), (int((weights != 0).sum()), loss)

    def train_step(self, weights, opt_state, optimizer):
        (loss, (l0, loss_)), grad = self.lwg(weights)

        updates, opt_state = optimizer.update(grad, opt_state, weights)
        weights = optax.apply_updates(weights, updates)

        return loss, weights, opt_state, dict(l0=l0, loss=loss_)

    def create_optimizer(self):
        optimizer = optax.chain(
            optax.adam(self.lr),
            optax.zero_nans(),
        )

        return optimizer

    def find_weights(self):
        weights = jnp.ones(self.sae["W_dec"].shape[0]) * self.init_w
        optimizer = self.create_optimizer()
        opt_state = optimizer.init(weights)

        min_loss = 1e9
        early_stopping_counter = 0

        for _ in (bar := trange(self.iterations)):
            loss, weights, opt_state, metrics = self.train_step(weights, opt_state, optimizer)

            if metrics["loss"] < min_loss:
                min_loss = metrics["loss"]
                early_stopping_counter = 0

            tk = jax.lax.top_k(weights, 2)

            bar.set_postfix(loss_optim=loss, **metrics, top=tk[1][0], top_diff=(tk[0][0] - tk[0][1]) / tk[0][0])

            early_stopping_counter += 1
            if early_stopping_counter > self.early_stopping_steps:
                break

        return weights, metrics
    
    def check_feature(self, feature, scale):
        steering_vector = self.sae["W_dec"][feature] * scale
        steering_vector =  steering_vector.astype('bfloat16')

        act_add = make_act_adder(
            llama, steering_vector, self.eval_tokens, self.target_layer
        )

        logits = act_add(self.eval_inputs).unwrap("batch", "seq", "vocabulary")

        return logprob_loss(logits, self.eval_tokens, n_first=self.n_first)

    def check_features(self, features, scale):
        losses = jnp.hstack([self.check_feature(feature, scale) for feature in tqdm(features)])

        return features[losses.argmin()], losses.min(), losses.mean(), losses

    

In [58]:
fs = FeatureSearch("en_es", 18, l1_coeff=5e-3, init_w=0.5, lr=5e-2)

In [59]:
w = fs.find_weights()

  0%|          | 0/2000 [00:00<?, ?it/s]

In [45]:
w

(# jax.Array float32(49152,) ≈-0.21 ±0.16 [≥-0.38, ≤1.1e+01] nonzero:49_152
   Array([-0.1067998 , -0.19951008, -0.13865103, ..., -0.23178048,
          -0.21337643, -0.2276923 ], dtype=float32)
 ,
 {'l0': 53, 'loss': <jax.Array(1.03125, dtype=bfloat16)>})

In [62]:
_, i = jax.lax.top_k(w[0], 20)

In [61]:
jax.lax.top_k(w[0], 20)

[# jax.Array float32(20,) ≈7.2 ±6.5 [≥-0.00092, ≤2.9e+01] nonzero:20
   Array([ 2.8505629e+01,  1.5451004e+01,  1.1832121e+01,  1.0702801e+01,
           1.0581120e+01,  9.6403446e+00,  8.9214010e+00,  8.9194613e+00,
           7.1935472e+00,  6.4322462e+00,  6.1946249e+00,  5.4800415e+00,
           4.7244701e+00,  3.8590069e+00,  1.3538325e+00,  1.2137506e+00,
           1.0922866e+00,  9.0917087e-01,  8.3833116e-01, -9.1777695e-04],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥764, ≤47_066] nonzero:20
   Array([  764, 12148, 44541, 26057,  5145, 24856,  7180, 28114, 44867,
          25612, 47066, 20311, 28406, 24695, 26165,  9035, 26756,  1312,
          13819, 22643], dtype=int32)
 ]

In [63]:
r = fs.check_features(i, 20)

  0%|          | 0/20 [00:00<?, ?it/s]

In [64]:
r


(<jax.Array(20311, dtype=int32)>,
 <jax.Array(6.5625, dtype=bfloat16)>,
 <jax.Array(12.125, dtype=bfloat16)>,
 # jax.Array bfloat16(20,) ≈1.2e+01 ±1.8 [≥6.6, ≤1.3e+01] nonzero:20
   Array([10.4375, 13.375, 13.125, 13.25, 13.125, 9.8125, 13.4375, 10.9375,
          12.9375, 13.4375, 11.8125, 6.5625, 9.625, 12.4375, 13.25, 13.0625,
          13.3125, 13.25, 12.0625, 13.25], dtype=bfloat16)
 )

In [17]:
import datasets
from datasets import load_dataset

dataset = load_dataset("Helsinki-NLP/opus-100", "en-es", split="validation")

In [22]:
sample = dataset.shuffle()[:20]
sample = sample["translation"]

In [23]:
pairs = [(x["en"], x["es"]) for x in sample]
pairs

[("You know it's just a spear, right?",
  'Sabes que es solo una lanza, ¿verdad?'),
 ('The lists of liqueurs appearing in the second subparagraph of paragraph 5 may be supplemented by the Council acting by a qualified majority on a proposal from the Commission.',
  'Las listas de los licores que figuran en el párrafo segundo del apartado 5 podrán ser completadas eventualmente por el Consejo, por mayoría cualificada y a propuesta de la Comisión.'),
 ('There shall be three security levels, as defined in Regulation (EC) No 725/2004:',
  'Existirán tres niveles de protección, según las definiciones del Reglamento (CE) no 725/2004:'),
 ('The desire of gypsies in each of the Member States to take their affairs into their own hands is becoming more and more evident.',
  'Cada día es más evidente el deseo de los gitanos de cada uno de los Estados miembros de la Comunidad de ser artífices de su propio destino.'),
 ("I hesitate to say this, and veer into PG-13, but you are the first person I've liked since...",
  'Dudo decir esto y convertirlo en inapropiado para menores de 13, pero eres la primera persona que me ha gustado desde...'),
 ('Germany regards Comprehensive Safeguards Agreements including the conclusion of an Additional Protocol as the safeguards standard that is demanded by Art. III of the Non-Proliferation Treaty.',
  'Alemania considera los Acuerdos de salvaguardias, incluida la conclusión de un Protocolo adicional, la norma de salvaguardias que se exige en el artículo III del Tratado sobre la no proliferación de las armas nucleares.'),
 ("Go to your room and think about all the toys you've got!",
  '¡Vete a tú habitación y piensa en toda la cantidad de juguetes que tienes!'),
 ('The Working Group further requested the Secretariat to review article XIII.4 of the GPA (which recognized the right of the procuring entity to ensure via enquiry of suppliers that they could comply with the conditions of participation and were capable of fulfilling the terms of the contract) and any proposed revised text, in order to take account of the approach set out in those provisions.',
  'El Grupo de Trabajo pidió además a la Secretaría que examinara el artículo XIII.4 del ACP (en que se reconoce el derecho de la entidad adjudicadora de cerciorarse, solicitando información a los proveedores, de que éstos reúnen las condiciones requeridas para participar en el proceso de adjudicación y serán capaces de cumplir las condiciones prescritas en el contrato) y toda revisión propuesta con objeto de tener en cuenta el criterio establecido en esas disposiciones.'),
 ('GOODS AND PEOPLE ARE LANDED HERE, WHICH IS NEW YORK CITY.',
  'Bienes y personas desembarcan aquí... la ciudad de Nueva York.'),
 ('Full-body CTs are awesome.',
  'La radiografía de cuerpo entero ha sido increíble.'),
 ("That's good, you got it.", 'Muy bien, los tengo.'),
 ('Was that part of the plan too?', '¿también era parte del plan?'),
 ('- Just behave yourself.', '- Comportate.'),
 ('Today, Jesus calls us to conversion, saying, "unless you repent you will all likewise perish" (Lk 3:3).',
  'Jesús nos llama hoy a la conversión y dice: "si no os arrepentís, todos pereceréis" (Lc 13, 3).'),
 ("Forgive me if I'm a bit blunt, but that's the sort of chap I am...",
  'Perdónenme si soy un poco brusco, pero yo soy asi.'),
 ("There's the wheelchair race of gloom and doom starring Boots Potter...",
  'Hay la carrera de silla de ruedas de la tristeza y la fatalidad protagonizada Botas Potter...'),
 ('He says, whoever talks will be swallowed up by the evil spirits of the rapids.',
  'Que quien hable será engullido por los espíritus malignos del remolino.'),
 ('Because I was born in this virile skin.',
  'Porque nací en esta piel viril.'),
 ("He's using a lot of hot water.", 'Utiliza un montón de agua caliente.'),
 ('- Well, yeah. - Uh-huh.', 'Bueno, sí.')]

In [46]:
fs = FeatureSearch("en_es", pairs, 18, l1_coeff=1e-2, init_w=0.5, lr=5e-2, n_shot=1, n_first=10)

In [47]:
w = fs.find_weights()

  0%|          | 0/2000 [00:00<?, ?it/s]

In [48]:
jax.lax.top_k(w[0], 20)

[# jax.Array float32(20,) ≈5.2 ±4.7 [≥-0.025, ≤1.5e+01] nonzero:20
   Array([15.284591  , 12.57232   , 12.362332  , 10.121915  ,  9.093476  ,
           7.3952093 ,  7.2686524 ,  6.7148376 ,  6.628763  ,  6.2520423 ,
           4.258019  ,  2.8919647 ,  1.0854002 ,  1.0788848 ,  0.98709387,
           0.34639516,  0.07082754, -0.01775918, -0.02440739, -0.02490301],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥5_145, ≤46_550] nonzero:20
   Array([20311, 34107, 24856, 46550,  5145, 31299, 12238, 16420, 28309,
          15057,  7774, 13563, 24669, 20375, 32081, 28142, 27475,  7459,
          35520, 27389], dtype=int32)
 ]

In [50]:
fs.check_feature(34107, 20)

<jax.Array(86, dtype=bfloat16)>

In [49]:
steering_vector = fs.sae["W_dec"][37312] * 40
steering_vector =  steering_vector.astype('bfloat16')

act_add = make_act_adder(
    llama, steering_vector, fs.eval_tokens, fs.target_layer, length=1
)


logits = act_add(fs.eval_inputs).unwrap("batch", "seq", "vocabulary")

In [50]:
positions = jnp.argwhere(fs.eval_tokens == 1599)[:, -1]


In [36]:
_logits = jax.vmap(
    lambda a, b: a[b]
)(logits, positions)
tokenizer.decode(_logits.argmax(axis=-1))

'down walk scene small hell scene ground film game sit method fe song enjoy definition in ent ag nation mad dog art tax oficial prote pages of life space golf union political'

In [37]:
fs.runner.eval_pairs

[[['down', 'abajo']],
 [['walk', 'caminar']],
 [['scene', 'escena']],
 [['large', 'grande']],
 [['hell', 'infierno']],
 [['scene', 'escena']],
 [['ground', 'tierra']],
 [['film', 'película']],
 [['stage', 'escenario']],
 [['sit', 'sentarse']],
 [['method', 'método']],
 [['pretty', 'bonito']],
 [['song', 'canción']],
 [['enjoy', 'disfrutar']],
 [['term', 'término']],
 [['into', 'en']],
 [['understand', 'comprender']],
 [['agency', 'agencia']],
 [['nation', 'nación']],
 [['hard', 'duro']],
 [['dog', 'perro']],
 [['art', 'arte']],
 [['tax', 'impuesto']],
 [['official', 'oficial']],
 [['protect', 'proteger']],
 [['page', 'página']],
 [['off', 'apagado']],
 [['life', 'vida']],
 [['space', 'espacio']],
 [['club', 'club']],
 [['union', 'unión']],
 [['political', 'político']]]

In [66]:
fs.eval_tokens

array([[32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13,  3204,  1599,   633,  7069],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 20919,  1599,  3949, 18220],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 24645,  1599,  3966,  2386],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13, 16961,  1599,  6815],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 14181,  1599,  3041, 12643],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 24645,  1599,  3966,  2386],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13,  2057,  1599, 27907],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13,  9663,  1599, 19053],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 19190,  1599,  3966, 24893],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13, 29879,   277,  1599,  2665,  7989],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13,  5696,  1599, 11510,  8144],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13,  1457,  4349,  1599, 10814,  2049],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13, 21453,  1599, 24174],
       [    1, 32010, 10306,   278,  4766, 29901,    13,   264,  2212,
        29891,  1599,   766,  1341,   329,   279],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13,  8489,  1599, 27757, 29877],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13,  8941,  1599,   427],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13,  5062,  1689,  1599,  7199,  1581],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13,   351,  3819,  1599,   946,  5760],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13, 29876,   362,  1599,  1055,  1290],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13,  6800,  1599,   868,   307],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 26169,  1599,   639,   307],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13,   442,  1599, 20160],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 20725,  1599,  2411, 29771],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 29877,  7880,  1599,  9535],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13, 14676,   312,  1599,  3279,   914],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13,  3488,  1599,   282, 18814],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13,  2696,  1599,  3095,   351,   912],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13, 19264,  1599, 10828],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13,  3493,  1599,  9015,  3934],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13, 29066,  1599,  4402],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 13094,  1599,   443,  3175],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 20087,   936,  1599, 24934]])

In [61]:
t

'down walked screen larger hello screen ground film stage sat method pu song enjoy value into understanding ag nation harder log art tax off protect pages off life space club union political'

In [38]:
def check_feature(feature: int, task:str, target_layer, sae, print_results=False):
    pairs = tasks[task]
    pairs = [list(x) for x in pairs.items()]
    dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=10, prepend_space=task.startswith("algo"))

    print(
        dataset.prompts
    )


    tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
    inputs = tokenized_to_inputs(
        **tokenized
    )

    pos = 1
    if task.startswith("algo"):
        pos = 2

    target_tokens = [x[pos] for x in tokenizer(dataset.completions)["input_ids"]]
    target_tokens = jnp.asarray(target_tokens)

    # weights, recon = grad_pursuit(tv * 2, sae["W_dec"], k, pos_only=True)

    recon = sae["W_dec"][feature] * 20
    recon = recon.astype('bfloat16')

    act_add = add_vector(
        llama, recon, target_layer, scale=1.0, position="last"
    )

    logits = act_add(inputs).unwrap("batch", "seq", "vocabulary")

    # print(logits[:, -1].mean(axis=-1))

    logprobs = jax.nn.log_softmax(logits, axis=-1)
    answer_logprobs = logprobs[:, -1]

    target_logprobs = jnp.take_along_axis(answer_logprobs, target_tokens[:, None], axis=-1).squeeze()

    if print_results:
        print(
            repr(tokenizer.decode(answer_logprobs.argmax(axis=-1)))
        )

        print(
            repr(tokenizer.decode(target_tokens))
        )

    return (target_logprobs - answer_logprobs.max(axis=-1))

In [39]:
sae = get_sae(17, 6)

check_feature(37312, "en_es", 17, sae, print_results=True)

['worry ->', 'money ->', 'hear ->', 'general ->', 'morning ->', 'government ->', 'course ->', 'letter ->', 'oil ->', 'future ->', 'kill ->', 'cut ->', 'risk ->', 'time ->', 'student ->', 'wear ->', 'organization ->', 'simply ->', 'population ->', 'doctor ->', 'attention ->', 'money ->', 'carry ->', 'require ->', 'court ->', 'fund ->', 'summer ->', 'something ->', 'environment ->', 'industry ->', 'reduce ->', 'player ->', 'money ->', 'subject ->', 'dead ->', 'comment ->', 'position ->', 'military ->', 'art ->', 'great ->', 'town ->', 'make ->', 'bill ->', 'fine ->', 'love ->', 'clear ->', 'drug ->', 'foreign ->', 'administration ->', 'discuss ->', 'watch ->', 'recognize ->', 'gun ->', 'gun ->', 'guy ->', 'rule ->', 'on ->', 'ok ->', 'club ->', 'much ->', 'situation ->', 'respond ->', 'cut ->', 'provide ->']
'wor mon o general tarde g cur number pet fut mat cort ries time estud vest organiz s población do at mon c require corte fund ver something natur indust redu jug mon verb dead comme

# jax.Array bfloat16(64,) ≈-2.5 ±3.2 [≥-1.4e+01, ≤0.0] zero:22 nonzero:42
  Array([-0.125, -3.5, -4.5625, 0, -1.74219, -0.1875, 0, -13.1875, -2.125,
         0, 0, 0, 0, -2.4375, 0, -3.125, 0, -2.625, 0, -1.1875, 0, -3.5,
         -1.11719, -7, 0, -0.878906, 0, -1.75781, -0.125, 0, 0, 0, -3.5,
         -6.1875, -3.375, -0.9375, -2.75, 0, -3.60938, -7.09375, -5.0625,
         -4.375, -11.0625, -13.5625, -2.5, 0, 0, -5.3125, -1.25, -0.375,
         -4.5, -0.125, -3.25, -3.25, 0, -6.0625, -6.5, 0, -0.4375, -0.25,
         -5.3125, 0, 0, -8.3125], dtype=bfloat16)

In [ ]:
for feature in i:
    print(
        f"Feature {feature}: {check_feature(feature)}"
    )

# get_logprob_diff(logits.unwrap("batch", "seq", "vocabulary"), target_tokens, print_results=True)

Feature 10018: -7.5
Feature 20311: -2.375
Feature 40023: -4.5
Feature 5145: -7.21875
Feature 45145: -7.09375
Feature 2719: -7.09375
Feature 15415: -7.28125
Feature 24802: -7.15625
Feature 20375: -7.125
Feature 19505: -7.09375
Feature 38618: -7.25
Feature 36786: -7.125
Feature 47611: -7.03125
Feature 31726: -7.28125


KeyboardInterrupt: 

In [ ]:
check_feature(764, True)

message reduce behavior place se hello cel media club live line throw sport es door shape daughter run pu
 bottom mind mat polit year moment cut hour has sit comment drop un government d wait phone anticip south arm sport material place picture shape papel no en < choice viel fine arrived ii f third pu hang u er must just hello fig
mess rid comport lu sic infer cell media club abit punto get sport così porta col fig corr bel reg super m question polit dec momento tag ora aver sed comment g gi governo pap asp tele asp sud es sport mater lu f col cart no met rela sc pi bene arriv io cin ter bel append fu dire do app infer figura


<jax.Array(-5.9375, dtype=bfloat16)>

In [17]:
fs = FeatureSearch("algo_last", 18)

--2024-05-23 13:29:53--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l18-test-run-6-1.01E-05/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.51, 108.156.211.95, 108.156.211.90, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.51|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/f057cb46f3d871ba03c66e707e3b3d8299322f36fa433862dc3fdca956715614?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716730193&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjczMDE5M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvZjA1N2NiNDZm

In [18]:
ds = fs.eval_dataset

In [17]:
info = """location_continent 21 -1.09375
football_player_position 21 -3.76562
location_religion 21 -1.09375
location_language 20 -0.457031
person_profession 21 -1.30469
location_country 21 -1.44531
country_capital 18 -1.25781
person_language 18 -0.287109
singular_plural 21 -0.261719
present_simple_past_simple 20 -0.341797
antonyms 14 -0.75
plural_singular 20 -0.133789
present_simple_past_perfect 18 -1.25781
present_simple_gerund 20 -0.171875
en_it 18 -1.29688
it_en 14 -1.75781
en_fr 18 -1.39062
en_es 18 -1.21875
fr_en 17 -1.25781
es_en 17 -1.09375
en_ru 18 ~1.1
en_de 18 ~1
algo_max 24 -0.515625
algo_min 13 -0.945312
algo_last 17 -0.162109
algo_first 18 -0.828125
algo_sum 14 -1.49219
algo_most_common 12 -1.15625
"""

In [18]:
tasks_to_check = [x.split()[:2] for x in info.split("\n") if x]
tasks_to_check = [(x[0], int(x[1])) for x in tasks_to_check]
tasks_to_check = [
    ("en_es", 18),
    ("en_it", 18),
    ("en_fr", 18),
    ("en_de", 18),
    ("en_ru", 18)
] 
tasks_to_check

[('en_es', 18), ('en_it', 18), ('en_fr', 18), ('en_de', 18), ('en_ru', 18)]

In [21]:
from datasets import load_dataset

results = {}

for task, target_layer in tqdm(tasks_to_check):
    if target_layer == 21:
        target_layer = 20
    if target_layer == 14:
        target_layer = 12
    if target_layer == 13:
        target_layer = 12
    # if target_layer == 17:
    #     target_layer = 18

    l,r = task.split("_")

    binding = f"{l}-{r}"
    if task == "en_de":
        binding = "de-en"

    dataset = load_dataset("Helsinki-NLP/opus-100", binding, split="validation")
    sample = dataset.shuffle()[:40]
    sample = sample["translation"]
    pairs = [(x[l], x[r]) for x in sample]

    fs = FeatureSearch(task, pairs, 18, l1_coeff=1e-2, init_w=0.5, lr=5e-2, n_shot=1, n_first=10, sae_v=8)
    weights, metrics = fs.find_weights()
    i = jax.lax.top_k(jnp.abs(weights), 20)[1]
    best_feature, best_loss, mean_loss, losses = fs.check_features(i, scale=20)

    metrics = {k:float(v) for k, v in metrics.items()}

    results[task] = (best_feature, best_loss, mean_loss, losses, metrics, i)
    print(
        f"Task {task}, best feature {best_feature}, best loss {best_loss}, mean loss {mean_loss}, metrics {metrics}"
    )

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Task en_es, best feature 22683, best loss 57.5, mean loss 63, metrics {'l0': 27.0, 'loss': 54.0}


  0%|          | 0/2000 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Task en_de, best feature 26059, best loss 71.5, mean loss 73.5, metrics {'l0': 14.0, 'loss': 68.5}


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Task en_ru, best feature 2431, best loss 68.5, mean loss 70, metrics {'l0': 110.0, 'loss': 63.75}


In [57]:
results

{'en_es': (<jax.Array(20311, dtype=int32)>,
  <jax.Array(51, dtype=bfloat16)>,
  <jax.Array(55.25, dtype=bfloat16)>,
  # jax.Array bfloat16(20,) ≈5.5e+01 ±1.6 [≥5.1e+01, ≤5.7e+01] nonzero:20
    Array([52.75, 51.5, 51, 54.25, 54.75, 55, 56, 54, 55.75, 56, 56, 56.5,
           56.25, 56, 56.5, 56.25, 56.25, 56, 56.75, 56], dtype=bfloat16)
  ,
  {'l0': 9.0, 'loss': 46.0},
  # jax.Array int32(20,) [≥764, ≤43_870] nonzero:20
    Array([  764, 28406, 20311, 24856,  7459, 10993, 43458, 40023, 22578,
           43870, 36865, 35465, 25316,  5824, 38413, 13641, 35664, 23702,
            9610, 19552], dtype=int32)
  ),
 'en_it': (<jax.Array(20412, dtype=int32)>,
  <jax.Array(65.5, dtype=bfloat16)>,
  <jax.Array(68, dtype=bfloat16)>,
  # jax.Array bfloat16(20,) ≈6.8e+01 ±0.98 [≥6.6e+01, ≤7e+01] nonzero:20
    Array([65.5, 67, 67, 67, 68, 67, 67.5, 67.5, 68.5, 70, 68.5, 69, 68, 68.5,
           68.5, 68.5, 69, 68.5, 69, 68.5], dtype=bfloat16)
  ,
  {'l0': 7.0, 'loss': 60.75},
  # jax.Array int32(20,) [≥823, ≤48_083] nonzero:20
    Array([20412, 24263, 26379,  7962, 43458,  7459, 22578, 28367,  7635,
           24168, 48083, 20510, 25592, 42682, 30523,   823, 22017, 39737,
           10918, 37866], dtype=int32)
  ),
 'en_fr': (<jax.Array(35638, dtype=int32)>,
  <jax.Array(56.25, dtype=bfloat16)>,
  <jax.Array(58.75, dtype=bfloat16)>,
  # jax.Array bfloat16(20,) ≈5.9e+01 ±0.79 [≥5.6e+01, ≤6e+01] nonzero:20
    Array([56.25, 57.5, 57.75, 58.75, 58.5, 59.5, 59.25, 59.25, 59, 58.75, 59,
           59, 59.25, 59.25, 59, 59.75, 59.25, 59.5, 59, 59], dtype=bfloat16)
  ,
  {'l0': 5.0, 'loss': 53.75},
  # jax.Array int32(20,) [≥1_972, ≤49_021] nonzero:20
    Array([35638, 32873, 40211, 43458, 22723, 28726, 24509, 20107, 34108,
           47062,  7989, 44165,  4125, 20264, 42133,  1972, 49021, 41964,
           25541, 29060], dtype=int32)
  )}

In [58]:
with open("results_s_s.pkl", "wb") as f:
    import pickle
    pickle.dump(results, f)

In [ ]:
import pickle

with open("results.pkl", "wb") as f:
    pickle.dump(results, f)
# with open("results.json", "w") as f:
#     json.dump(results, f)

In [26]:
for task, (best_feature, best_loss, mean_loss, losses, metrics) in results.items():
    print(
        f"{task}, best feature {best_feature}, loss ratio {mean_loss / best_loss}, loss {best_loss}"
    )

algo_max, best feature 28655, loss ratio 1.61719, loss -0.816406
algo_min, best feature 42195, loss ratio 1.14062, loss -1.3125
algo_last, best feature 42756, loss ratio 1.46094, loss -0.839844
algo_first, best feature 39302, loss ratio 11.6875, loss -0.0683594
algo_sum, best feature 10061, loss ratio 1.22656, loss -1.08594
algo_most_common, best feature 3788, loss ratio 1.34375, loss -0.933594


In [ ]:
fs = FeatureSearch("en_de", 17)

--2024-05-22 00:38:44--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l17-test-run-6-4.52E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.90, 108.156.211.95, 108.156.211.125, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/1623d8da38be3171fcc8516a4cbe9fdb80e3d77e370aa5690895697649d688f3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716597524&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjU5NzUyNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvMTYyM2Q4ZGE

In [20]:
fs = FeatureSearch("en_ru", 18, early_stopping_steps=100, seed=10)
w = fs.find_weights()

--2024-05-22 01:20:39--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l18-test-run-6-1.01E-05/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.51, 108.156.211.90, 108.156.211.95, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.51|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/f057cb46f3d871ba03c66e707e3b3d8299322f36fa433862dc3fdca956715614?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716600039&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjYwMDAzOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvZjA1N2NiNDZm

  0%|          | 0/2000 [00:00<?, ?it/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f12cc5bf290>>
Traceback (most recent call last):
  File "/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.12/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [32]:
fs = FeatureSearch("algo_last", 18, early_stopping_steps=100, seed=10)

In [33]:
w = fs.find_weights()

  0%|          | 0/2000 [00:00<?, ?it/s]

In [21]:
jax.lax.top_k(w[0], 20)

[# jax.Array float32(20,) ≈8.6 ±1.1e+01 [≥-0.055, ≤3.1e+01] nonzero:20
   Array([ 3.09979153e+01,  2.82771473e+01,  2.81605225e+01,  2.23171616e+01,
           1.70592213e+01,  1.41996422e+01,  1.22735662e+01,  1.18328285e+01,
           3.67472243e+00,  2.47404289e+00,  1.58222604e+00, -2.69895289e-02,
          -2.70592347e-02, -2.96332240e-02, -3.23766917e-02, -4.17804755e-02,
          -4.86377254e-02, -5.33655584e-02, -5.33791259e-02, -5.49723543e-02],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥6_262, ≤46_627] nonzero:20
   Array([10615, 41937, 10151, 13937, 16245, 46627,  6262, 16804,  8286,
          25117, 10897, 28142, 21109, 21014, 44885, 43632, 44120,  7628,
          25342, 36901], dtype=int32)
 ]

In [34]:
fs.check_features(jax.lax.top_k(w[0], 20)[1], scale=20)

  0%|          | 0/20 [00:00<?, ?it/s]

(<jax.Array(21195, dtype=int32)>,
 <jax.Array(-0.162109, dtype=bfloat16)>,
 <jax.Array(-0.980469, dtype=bfloat16)>,
 # jax.Array bfloat16(20,) ≈-0.98 ±0.37 [≥-1.4, ≤-0.16] nonzero:20
   Array([-0.162109, -0.546875, -0.330078, -1.24219, -0.757812, -0.53125,
          -0.390625, -1.14844, -1.19531, -1.10156, -1.32812, -1.28125,
          -1.14844, -1.21875, -1.35938, -1.02344, -1.16406, -1.24219,
          -0.980469, -1.42188], dtype=bfloat16)
 )

In [ ]:
fs.check_features(jax.lax.top_k(w[0], 20)[1], scale=20)

  0%|          | 0/20 [00:00<?, ?it/s]

(<jax.Array(41239, dtype=int32)>,
 <jax.Array(-4.21875, dtype=bfloat16)>,
 <jax.Array(-5.34375, dtype=bfloat16)>,
 # jax.Array bfloat16(20,) ≈-5.3 ±0.37 [≥-6.0, ≤-4.2] nonzero:20
   Array([-6, -5.28125, -5.65625, -5.15625, -5.03125, -5.15625, -5.53125,
          -5.25, -5.28125, -5.65625, -5.1875, -5.71875, -5.59375, -5.0625,
          -4.21875, -5.40625, -5.0625, -5.53125, -5.21875, -5.65625],      dtype=bfloat16)
 )

In [ ]:
jax.lax.top_k(w, 20)

[# jax.Array float32(20,) ≈5.4 ±2.9 [≥-0.014, ≤1.1e+01] nonzero:20
   Array([11.022304  ,  9.960602  ,  9.723273  ,  7.446827  ,  7.3656707 ,
           6.9718046 ,  6.634519  ,  6.3051505 ,  6.268656  ,  6.1738625 ,
           5.3788037 ,  4.8796234 ,  4.4844356 ,  4.196526  ,  3.5582683 ,
           2.7230783 ,  1.9743043 ,  1.7151848 ,  1.7031406 , -0.01442439],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥3_151, ≤47_066] nonzero:20
   Array([ 9644, 20311, 46157, 42589, 25117, 38261,  9337, 47066, 10897,
          30124,  3151,  3398, 46627, 36916, 17215, 10069, 29942, 39270,
           4999, 31578], dtype=int32)
 ]

In [ ]:
def find_features(task, target_layer):
    scale = 20
    early_stopping_steps = 50
    init_w = 0.1
    iterations = 2000

    sae = get_sae(target_layer, 6)
    dictionary = sae["W_dec"]

    pairs = tasks[task]
    pairs = [list(x) for x in pairs.items()]
    seed = 0

    dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=seed+1)


    tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
    inputs = tokenized_to_inputs(
        **tokenized
    )

    target_tokens = [x[1] for x in tokenizer(dataset.completions)["input_ids"]]
    target_tokens = jnp.asarray(target_tokens)

    jittify = lambda x: partial(jax.jit(lambda lr, *args, **kwargs: lr(*args, **kwargs)[1][0].value), x)
    get_resids_initial = make_get_resids(llama, target_layer)
    get_resids_initial = jittify(get_resids_initial)

    intital_resids = get_resids_initial(inputs)

    taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
        lambda i, x: x if i >= target_layer else pz.nn.Identity()
    ).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                    .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

    optimizer = optax.chain(
        optax.adam(3e-2),
        optax.zero_nans(),    
    )

    lwg = jax.value_and_grad(get_loss, has_aux=True)
    

    weights = jnp.ones(dictionary.shape[0]) * init_w
    opt_state = optimizer.init(weights)

    min_loss = 1e9
    early_stopping_counter = 0

    for _ in (bar := trange(iterations)):
        loss, weights, opt_state, metrics = train_step(weights, opt_state, dictionary, inputs, target_tokens, taker, intital_resids, optimizer, lwg)

        if metrics["loss"] < min_loss:
            min_loss = metrics["loss"]
            early_stopping_counter = 0
        

        tk = jax.lax.top_k(weights, 2)

        bar.set_postfix(loss_optim=loss, **metrics, top=tk[1][0], top_diff=(tk[0][0] - tk[0][1]) / tk[0][0])

        early_stopping_counter += 1
        if early_stopping_counter > early_stopping_steps:
            break

    
    return jax.lax.top_k(jnp.abs(weights), 20)


In [ ]:
find_features(
    "en_es", 18
)

  0%|          | 0/2000 [00:00<?, ?it/s]

[# jax.Array float32(20,) ≈3.3 ±4.8 [≥0.42, ≤1.6e+01] nonzero:20
   Array([16.070333  , 12.78331   , 10.591674  , 10.247323  ,  5.344763  ,
           2.8526528 ,  1.5353229 ,  0.5156362 ,  0.50892174,  0.45389727,
           0.45202777,  0.44730943,  0.44149902,  0.43828565,  0.4329766 ,
           0.43184933,  0.43073234,  0.4306618 ,  0.42252657,  0.4161794 ],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥2_719, ≤48_025] nonzero:20
   Array([10018, 20311,  5145, 40023, 45145,  2719, 15415, 24802, 20375,
          19505, 38618, 36786, 47611, 31726, 12632, 10615, 47480, 48025,
           3048, 40850], dtype=int32)
 ]

In [35]:
sae = get_sae(target_layer, 6)

--2024-05-22 18:45:26--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l17-test-run-6-4.52E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.125, 108.156.211.90, 108.156.211.95, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.125|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/1623d8da38be3171fcc8516a4cbe9fdb80e3d77e370aa5690895697649d688f3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716662726&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjY2MjcyNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvMTYyM2Q4ZG

In [36]:
check_feature(21195, "algo_last", 18, sae)

NameError: name 'check_feature' is not defined